In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./kit_part.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_ = input_.drop_duplicates(['Kit_Info'], keep='first').reset_index(drop=True)

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get(input_.loc[a, 'Kit_Info'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp)
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =
                
                list_tr = html.xpath('//table[@class="moreinfotable"]/tr')[1:]
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'Kit_Info': input_.loc[a, 'Kit_Info']}])

                for tr in list_tr:
                    df_temp.loc[0, tr.xpath('./td[1]')[0].text.lower().strip()] = tr.xpath('./td[2]')[0].text.strip()

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Kit_Info': input_.loc[a, 'Kit_Info']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Kit_Info'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(10):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Kit_Info'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./kit_info-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Kit_Info'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./kit_info_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：890

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5724&pk=8306420
[尝试次数：1] - [剩余数量：880] - [当前时间：14:18:13]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5724&pk=10110008
[尝试次数：1] - [剩余数量：879] - [当前时间：14:18:13]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=10110020
[尝试次数：1] - [剩余数量：878] - [当前时间：14:18:13]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5736&pk=10110032
[尝试次数：1] - [剩余数量：877] - [当前时间：14:18:13]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5736&pk=10110024
[尝试次数：1] - [剩余数量：876] - [当前时间：14:18:13]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=10110016
[尝试次数：1] - [剩余数量：875] - [当前时间：14:18:14]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=10465&pk=10195124
[尝试次数：1] - [剩余数量：874] - [当前时间：14:18:14]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=10110040
[尝试次数：1] - [剩余数量：873] - [当前时间：14:18:14]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5723&pk=8314716
[尝试次数：1] - [剩余数量：872] - [当前时间：14:18:14]

[ok] - http

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5724&pk=8304556
[尝试次数：2] - [剩余数量：802] - [当前时间：14:18:33]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5724&pk=8304560
[尝试次数：3] - [剩余数量：801] - [当前时间：14:18:34]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=8308292
[尝试次数：14] - [剩余数量：800] - [当前时间：14:18:34]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=8307536
[尝试次数：1] - [剩余数量：799] - [当前时间：14:18:34]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5736&pk=8308252
[尝试次数：1] - [剩余数量：798] - [当前时间：14:18:35]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5736&pk=8308220
[尝试次数：5] - [剩余数量：797] - [当前时间：14:18:35]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5736&pk=8308256
[尝试次数：1] - [剩余数量：796] - [当前时间：14:18:35]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=8307528
[尝试次数：5] - [剩余数量：795] - [当前时间：14:18:35]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5736&pk=8308248
[尝试次数：3] - [剩余数量：794] - [当前时间：14:18:36]

[ok] - https://www.rockauto

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=8307040
[尝试次数：5] - [剩余数量：726] - [当前时间：14:19:00]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=8307488
[尝试次数：2] - [剩余数量：725] - [当前时间：14:19:00]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5724&pk=8306484
[尝试次数：3] - [剩余数量：724] - [当前时间：14:19:00]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=8307028
[尝试次数：4] - [剩余数量：723] - [当前时间：14:19:01]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=11158173
[尝试次数：9] - [剩余数量：722] - [当前时间：14:19:01]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5723&pk=8314724
[尝试次数：6] - [剩余数量：721] - [当前时间：14:19:01]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5723&pk=11492621
[尝试次数：2] - [剩余数量：720] - [当前时间：14:19:01]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=8307036
[尝试次数：1] - [剩余数量：719] - [当前时间：14:19:02]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=11492521
[尝试次数：1] - [剩余数量：718] - [当前时间：14:19:02]

[ok] - https://www.rockau

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=13570101
[尝试次数：5] - [剩余数量：649] - [当前时间：14:19:27]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5724&pk=8306460
[尝试次数：4] - [剩余数量：648] - [当前时间：14:19:27]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5723&pk=8314944
[尝试次数：2] - [剩余数量：647] - [当前时间：14:19:27]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5756&pk=10981368
[尝试次数：10] - [剩余数量：646] - [当前时间：14:19:27]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=8307524
[尝试次数：5] - [剩余数量：645] - [当前时间：14:19:27]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5564&pk=8310240
[尝试次数：1] - [剩余数量：644] - [当前时间：14:19:29]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5723&pk=8314816
[尝试次数：3] - [剩余数量：643] - [当前时间：14:19:29]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5564&pk=8310396
[尝试次数：1] - [剩余数量：642] - [当前时间：14:19:29]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5724&pk=8306424
[尝试次数：2] - [剩余数量：641] - [当前时间：14:19:29]

[ok] - https://www.rockau

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=8527976
[尝试次数：4] - [剩余数量：573] - [当前时间：14:19:54]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5724&pk=14047597
[尝试次数：3] - [剩余数量：572] - [当前时间：14:19:54]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5568&pk=14047601
[尝试次数：4] - [剩余数量：571] - [当前时间：14:19:55]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=10465&pk=15383821
[尝试次数：15] - [剩余数量：570] - [当前时间：14:19:55]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5724&pk=14047613
[尝试次数：3] - [剩余数量：569] - [当前时间：14:19:55]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5723&pk=9659224
[尝试次数：1] - [剩余数量：568] - [当前时间：14:19:55]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5736&pk=9659116
[尝试次数：1] - [剩余数量：567] - [当前时间：14:19:56]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5736&pk=9659120
[尝试次数：2] - [剩余数量：566] - [当前时间：14:19:56]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5736&pk=9659124
[尝试次数：3] - [剩余数量：565] - [当前时间：14:19:56]

[ok] - https://www.roc

...

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5723&pk=8314464
[尝试次数：2] - [剩余数量：343] - [当前时间：14:21:13]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5723&pk=7084420
[尝试次数：3] - [剩余数量：342] - [当前时间：14:21:13]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=8307416
[尝试次数：1] - [剩余数量：341] - [当前时间：14:21:13]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5724&pk=8304696
[尝试次数：5] - [剩余数量：340] - [当前时间：14:21:14]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5736&pk=8308124
[尝试次数：3] - [剩余数量：339] - [当前时间：14:21:15]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5723&pk=8314880
[尝试次数：2] - [剩余数量：338] - [当前时间：14:21:15]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=11859&pk=8921272
[尝试次数：6] - [剩余数量：337] - [当前时间：14:21:15]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=8307412
[尝试次数：4] - [剩余数量：336] - [当前时间：14:21:15]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5188&pk=8306472
[尝试次数：1] - [剩余数量：335] - [当前时间：14:21:16]

[ok] - https://www.rockauto

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=8307252
[尝试次数：1] - [剩余数量：267] - [当前时间：14:21:42]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=8307256
[尝试次数：1] - [剩余数量：266] - [当前时间：14:21:42]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5723&pk=8314600
[尝试次数：3] - [剩余数量：265] - [当前时间：14:21:42]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=8307856
[尝试次数：1] - [剩余数量：264] - [当前时间：14:21:43]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=8307284
[尝试次数：1] - [剩余数量：263] - [当前时间：14:21:43]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5724&pk=8305944
[尝试次数：1] - [剩余数量：262] - [当前时间：14:21:43]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5736&pk=8307860
[尝试次数：4] - [剩余数量：261] - [当前时间：14:21:44]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=7096488
[尝试次数：1] - [剩余数量：260] - [当前时间：14:21:44]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5722&pk=8314564
[尝试次数：1] - [剩余数量：259] - [当前时间：14:21:44]

[ok] - https://www.rockauto.

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=8307248
[尝试次数：1] - [剩余数量：189] - [当前时间：14:22:14]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5736&pk=8307848
[尝试次数：5] - [剩余数量：188] - [当前时间：14:22:14]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5724&pk=8306152
[尝试次数：3] - [剩余数量：187] - [当前时间：14:22:15]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=10709&pk=8307868
[尝试次数：9] - [剩余数量：186] - [当前时间：14:22:15]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5722&pk=8314796
[尝试次数：1] - [剩余数量：185] - [当前时间：14:22:15]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=8307384
[尝试次数：4] - [剩余数量：184] - [当前时间：14:22:16]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=10465&pk=10195116
[尝试次数：1] - [剩余数量：183] - [当前时间：14:22:16]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5723&pk=10195108
[尝试次数：1] - [剩余数量：182] - [当前时间：14:22:16]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5722&pk=10195112
[尝试次数：1] - [剩余数量：181] - [当前时间：14:22:17]

[ok] - https://www.rock

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5736&pk=8308156
[尝试次数：1] - [剩余数量：113] - [当前时间：14:22:39]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5724&pk=8306436
[尝试次数：2] - [剩余数量：112] - [当前时间：14:22:39]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=8307456
[尝试次数：1] - [剩余数量：111] - [当前时间：14:22:40]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5722&pk=8313596
[尝试次数：5] - [剩余数量：110] - [当前时间：14:22:40]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=10709&pk=8921332
[尝试次数：1] - [剩余数量：109] - [当前时间：14:22:40]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=8307452
[尝试次数：3] - [剩余数量：108] - [当前时间：14:22:40]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5723&pk=8314904
[尝试次数：4] - [剩余数量：107] - [当前时间：14:22:40]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=8308160
[尝试次数：2] - [剩余数量：106] - [当前时间：14:22:40]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=51028&pk=9659032
[尝试次数：2] - [剩余数量：105] - [当前时间：14:22:41]

[ok] - https://www.rockaut

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5724&pk=9659036
[尝试次数：1] - [剩余数量：36] - [当前时间：14:23:07]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=9659080
[尝试次数：1] - [剩余数量：35] - [当前时间：14:23:07]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=8921296
[尝试次数：7] - [剩余数量：34] - [当前时间：14:23:07]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=9659140
[尝试次数：1] - [剩余数量：33] - [当前时间：14:23:08]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5724&pk=9659052
[尝试次数：1] - [剩余数量：32] - [当前时间：14:23:08]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5188&pk=8306444
[尝试次数：1] - [剩余数量：31] - [当前时间：14:23:08]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=9659084
[尝试次数：6] - [剩余数量：30] - [当前时间：14:23:09]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=8921292
[尝试次数：11] - [剩余数量：29] - [当前时间：14:23:09]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=10709&pk=8308196
[尝试次数：2] - [剩余数量：28] - [当前时间：14:23:10]

[ok] - https://www.rockauto.com/en/